In [1]:
import pandas as pd
import pickle
import tensorflow as tf

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, LSTM, Dense, Activation, Conv1D, Flatten, Dropout, MaxPooling1D

import sys
sys.path.append('../../..')
from smartFAQ.src.tokenPad import tokenization_padding

import os

print('Setup is ready')

Setup is ready


# Load Data
---
## Fast_text model

In [2]:
# with open('../../data/fast_text_model_pretrained_weights.pickle', 'rb') as handle:
#     fast_text_model = pickle.load(handle)
with open('../../data/x_train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
with open('../../data/y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
with open('../../data/x_val.pickle', 'rb') as handle:
    X_val = pickle.load(handle)
with open('../../data/y_val.pickle', 'rb') as handle:
    y_val = pickle.load(handle)

## Tokenization & Padding

In [3]:
MAX_NUM_WORDS = 20000

In [4]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(X_train['answer'])
input_integer_seq = input_tokenizer.texts_to_sequences(X_train['answer'])

fast_text_idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(fast_text_idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 30369
Length of longest sentence in input: 1688


In [5]:
x_train_pad = pad_sequences(input_integer_seq, maxlen=max_input_len)

print("encoder_input_sequences.shape:", x_train_pad.shape)
print("encoder_input_sequences[172]:", x_train_pad[72])

encoder_input_sequences.shape: (10000, 1688)
encoder_input_sequences[172]: [  0   0   0 ... 135 183  86]


In [6]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(X_val['answer'])
input_integer_seq = input_tokenizer.texts_to_sequences(X_val['answer'])
x_val_pad = pad_sequences(input_integer_seq, maxlen=max_input_len)

print("encoder_input_sequences.shape:", x_val_pad.shape)
print("encoder_input_sequences[172]:", x_val_pad[72])

encoder_input_sequences.shape: (10000, 1688)
encoder_input_sequences[172]: [   0    0    0 ... 5837 5419 5838]


## Keras Model

In [7]:
import gensim
from gensim.models import FastText
from gensim.models.phrases import Phrases, Phraser


In [ ]:
# sent = []
# for col in ['question', 'answer']:
#     words = [row.split() for row in X_train[col]]
#     sent = sent + words
# sentences = [[word for word in document.lower().split()] for document in X_train['answer']]

In [ ]:
#sentences = [[word for word in document.lower().split()] for document in X_train['answer']]

In [ ]:
## Bigrams

#phrases = Phrases(sentences)
#bigram = Phraser(phrases)
#sentences = bigram[sentences]

In [ ]:
# fast_text_model = FastText(min_count=50, window=10, size=300, sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=50, workers=cores-1)

In [ ]:
# sentences

In [8]:
with open('../../data/fast_text_model_pretrained_weights.pickle', 'rb') as handle:
    fast_text_model = pickle.load(handle)

In [9]:
pretrained_weights = fast_text_model.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape

C:\Users\achenu\Anaconda3\envs\FAQ\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [10]:
fast_text_model.most_similar(['import'])

C:\Users\achenu\Anaconda3\envs\FAQ\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('import_sys', 0.7419381141662598),
 ('import_os', 0.7078060507774353),
 ('Import', 0.7003344893455505),
 ('import_glob', 0.6803531646728516),
 ('imports', 0.6776682138442993),
 ('imported', 0.6736365556716919),
 ('bs_import', 0.6357274055480957),
 ('flask_import', 0.6279746294021606),
 ('import_threading', 0.6251416206359863),
 ('future_import', 0.6239302754402161)]

# Build

In [11]:
def build_model(vocab_size,embedding_size,pretrained_weights):
    
           
    model = tf.keras.Sequential()
    
    model.add(Embedding(input_dim=vocab_size, 
                        output_dim=embedding_size, 
                        weights=[pretrained_weights],
                        input_length=max_input_len
                       ))

    model.add(LSTM(units=embedding_size))
    model.add(Dense(units=vocab_size))
    model.add(Activation('softmax'))
    model.add(Dense(1))

    return model

In [12]:
def compile_model(model):
    mse = tf.keras.losses.MeanSquaredError()
    model.compile(loss=mse,
                  optimizer='adam',
                  metrics=['MeanSquaredError']
                  )
    return  model

In [13]:
def fit_model( x_train, y_train, x_val, y_val, model, batch_size,  epochs = 5):
    
    print('Train...')
    os.makedirs("./logs",exist_ok=True)
    tensorboard = TensorBoard(log_dir=os.path.join('./logs'), histogram_freq=0,
                                  write_graph=True, write_images=False,profile_batch = 100000000)

    callbacks = [ tensorboard]

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_val, y_val),
              callbacks= callbacks)
    return model

In [14]:
with tf.device("/CPU:0"):
    model = build_model(vocab_size, embedding_size, pretrained_weights)
    model.summary()
    model = compile_model(model)
    model = fit_model(x_train_pad, y_train, x_val_pad, y_val, model, batch_size=200, epochs=10)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1688, 300)         1131600   
_________________________________________________________________
lstm (LSTM)                  (None, 300)               721200    
_________________________________________________________________
dense (Dense)                (None, 3772)              1135372   
_________________________________________________________________
activation (Activation)      (None, 3772)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3773      
Total params: 2,991,945
Trainable params: 2,991,945
Non-trainable params: 0
_________________________________________________________________
Train...
Epoch 1/10


InvalidArgumentError:  indices[32,1582] = 8799 is not in [0, 3772)
	 [[node sequential/embedding/embedding_lookup (defined at <ipython-input-13-23f7722d067e>:14) ]] [Op:__inference_train_function_2783]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/embedding/embedding_lookup:
 sequential/embedding/embedding_lookup/1811 (defined at C:\Users\achenu\Anaconda3\envs\FAQ\lib\contextlib.py:112)

Function call stack:
train_function


In [15]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('./saved_model/LSTM_model_FT')

Un sous-r‚pertoire ou un fichier -p existe d‚j….
Une erreur s'est produite lors du traitement deÿ: -p.
Un sous-r‚pertoire ou un fichier saved_model existe d‚j….
Une erreur s'est produite lors du traitement deÿ: saved_model.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./saved_model/LSTM_model_FT\assets


In [16]:
(loss, acc) = model.evaluate(x_val_pad, y_val)

InvalidArgumentError:  indices[31,1651] = 10483 is not in [0, 3772)
	 [[node sequential/embedding/embedding_lookup (defined at <ipython-input-16-2bc3015b0f35>:1) ]] [Op:__inference_test_function_9965]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/embedding/embedding_lookup:
 sequential/embedding/embedding_lookup/9461 (defined at C:\Users\achenu\Anaconda3\envs\FAQ\lib\contextlib.py:112)

Function call stack:
test_function


In [ ]:
# © Laëtitia CONSTANTIN & Axel CHENU 2021